In [ ]:
import pandas as pd
import numpy as np
import os

# visualize
import seaborn as sns


# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

# acquire
from pydataset import data

In [ ]:
# *************************************  connection url **********************************************

# Create helper function to get the necessary connection url.
def get_connection(db_name):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    from env import host, username, password
    return f'mysql+pymysql://{username}:{password}@{host}/{db_name}'


In [ ]:
# acquire data

In [ ]:
#acquire data for the first time
def get_new_telco_churn():
    '''
    This function reads in the telco_churn data from the Codeup db
    and returns a pandas DataFrame with all columns and joined with other tables.
    '''
    sql_query = '''
    SELECT * FROM customers
    JOIN contract_types USING (contract_type_id)
    JOIN internet_service_types USING (internet_service_type_id)
    JOIN payment_types USING (payment_type_id)
    '''
    return pd.read_sql(sql_query, get_connection('telco_churn'))


In [ ]:
#acquire data 
def get_telco_churn():
    '''
    This function reads in telco_churn data from Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('telco_churn.csv'):
        
        # If csv file exists, read in data from csv file.
        df = pd.read_csv('telco_churn.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame.
        df = get_new_telco_churn()
        
        # Write DataFrame to a csv file.
        df.to_csv('telco_churn.csv')
        
    return df

In [ ]:
df= get_telco_churn()

In [ ]:
# check the information before preparation process
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

# preparation 

In [ ]:
df.isnull().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
#we cannot conver total_charges to float because there are ' '
#df['total_charges'].astype('float')

In [ ]:
#take a look. these are new customers 
df['total_charges'][df['total_charges']== ' '] 

In [ ]:
df['total_charges'][df['total_charges']== ' '] + '0'

In [ ]:
df['total_charges'][df['total_charges']== ' '] = df['total_charges'][df['total_charges']== ' '] + '0'

In [ ]:
#in order to conver total_charges to float, I need will add '0' to ' '
#df['total_charges'] = df['total_charges'] + '0'

In [ ]:
#convert to a float type
df['total_charges'] = df['total_charges'].astype('float')
df['total_charges'].dtype

In [ ]:
#checking for duplicates
df.duplicated().sum()

In [ ]:
df.info()

In [ ]:
df.head(1)

In [ ]:
#all the columns that have yes/no
col_list = ['partner', 'dependents','phone_service',  #'tech_support', 'streaming_tv','streaming_movies'
            'paperless_billing','churn' ]

In [ ]:
#check all the values for these columns
for col in col_list:
    print(col)
    print(df[col].value_counts())
    print('__________________________')

In [ ]:
(df.churn == 'Yes').astype(int).value_counts()

In [ ]:
(df[col_list] == 'Yes').astype(int)

In [ ]:
df[col_list] = (df[col_list] == 'Yes').astype(int)

In [ ]:
df.head()

In [ ]:
#check all the values for these columns to make sure it is correct
for col in col_list:
    print(col)
    print(df[col].value_counts(dropna = False))
    print('__________________________')

In [ ]:
df.info()

In [ ]:
df.select_dtypes('object').columns

In [ ]:
list(df.select_dtypes('object').columns)[1:]

In [ ]:
col_list = list(df.select_dtypes('object').columns)[1:]
for col in col_list:
    print(col)
    print(df[col].value_counts())
    print('__________________________')

In [ ]:
col_list[2:8]

In [ ]:
#create a dicttionary to change some columns to 0,1,2,
var= {
    'No':0,
    'Yes':1,
    'No internet service':3
}

In [ ]:
#using .map to change the values 

In [ ]:
for col in col_list[2:8]:
      df[col]= df[col].map(var) 


In [ ]:
#check the changes
df[col_list[2:8]].head()

In [ ]:
list(df.select_dtypes('object').columns)

In [ ]:
df['multiple_lines'].unique()

In [ ]:
df.replace({'multiple_lines': {'No':0, 'Yes':1, 'No phone service': 2}}, inplace=True)


In [ ]:
df['multiple_lines'].unique()

In [ ]:
df.corr()

In [ ]:
# I can create dummies for gender and contract_type

In [ ]:
#creating my function

def clean_data(df, dummies):
    '''
    This function will drop any duplicate observations, 
    drop ['deck', 'embark_town', 'class'], fill missing embarked with 'Southampton'
    and create dummy vars from sex and embarked. 
    '''

    #clean data
    # conver total_charges to float, first I need will add '0' to ' '
    df['total_charges'] = df['total_charges'] + '0'
    df['total_charges'] = df['total_charges'].astype('float')
    
    #converr all the columns that have yes/no to 0/1
    col_list = ['partner', 'dependents','phone_service', 'tech_support', 'streaming_tv',
            'streaming_movies', 'paperless_billing','churn' ]
    df[col_list] = (df[col_list] == 'Yes').astype(int)
    
    
    #create a dummy df
    dummy_df = pd.get_dummies(df[dummies], drop_first=[True, True])
    ## Concatenate the dummy_df dataframe above with the original df
    df = pd.concat([df, dummy_df], axis=1)
    # drop the columns that we already use to create dummy_df
    df = df.drop(columns= dummies)
    
    return df
